In [1]:
import requests as reqs 
import copy
import json
import os
import pandas as pd
import numpy as np
import glob

import os
import re

import seaborn as sns

%matplotlib inline
import matplotlib.pyplot as plt
import plotly.express as px

import datetime
from datetime import timedelta
from pandas.api.types import is_datetime64_any_dtype as is_datetime

import sklearn

from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import MinMaxScaler, StandardScaler

In [2]:
data_clean = 'data/clean'
parent = '/Users/ilonanietosvaara/Documents/Kurssit/spiced/spiced_projects/scikit-cilantro-student-code/final_project'

datapath_clean = os.path.join(parent, data_clean)

In [3]:
os.chdir(datapath_clean)

In [4]:
df = pd.read_csv(f'EDA_data.csv', index_col=None)

In [5]:
df.shape

(29619, 39)

In [6]:
df

,act_player_id,catch,drop,goalcatch,callahan,d,pull,callahan_throw,throw,dropped_throw,...,tournamemnt,game_id_str,season,date_id,act_player,throws_all,turnover,pass_perc,catch_perc,ranking2019
0,chatte_s_0,16.0,0.0,2.0,0.0,1.0,0.0,0.0,12.0,0.0,...,AUDL West 2015,2015-04-11 18:21_SanFranciscoFlameThrowers2015-stats.csv,2015.0,0.0,chatte_s,16.0,1.0,0.937500,1.0,0
1,chialv_f_0,27.0,0.0,0.0,0.0,1.0,2.0,0.0,45.0,0.0,...,AUDL West 2015,2015-04-11 18:21_SanFranciscoFlameThrowers2015-stats.csv,2015.0,0.0,chialv_f,50.0,3.0,0.940000,1.0,0
2,dallma_l_0,18.0,0.0,4.0,0.0,1.0,0.0,0.0,12.0,0.0,...,AUDL West 2015,2015-04-11 18:21_SanFranciscoFlameThrowers2015-stats.csv,2015.0,0.0,dallma_l,17.0,4.0,0.764706,1.0,0
3,fein_b_0,5.0,0.0,0.0,0.0,1.0,0.0,0.0,5.0,0.0,...,AUDL West 2015,2015-04-11 18:21_SanFranciscoFlameThrowers2015-stats.csv,2015.0,0.0,fein_b,5.0,0.0,1.000000,1.0,0
4,ikeda_c_0,10.0,0.0,0.0,NaN,NaN,NaN,0.0,11.0,1.0,...,AUDL West 2015,2015-04-11 18:21_SanFranciscoFlameThrowers2015-stats.csv,2015.0,0.0,ikeda_c,13.0,0.0,1.000000,1.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29614,perivier_1503,NaN,NaN,NaN,NaN,NaN,NaN,0.0,1.0,0.0,...,NaN,2019-06-29 21:29_AtlantaHustle2019-stats.csv,2019.0,224.0,perivier,1.0,0.0,1.000000,NaN,11
29615,brodbeck_1507,NaN,NaN,NaN,NaN,NaN,NaN,0.0,1.0,0.0,...,NaN,2019-06-30 13:09_AustinSol2019-stats.csv,2019.0,225.0,brodbeck,1.0,0.0,1.000000,NaN,18
29616,mika_c_1507,NaN,NaN,NaN,NaN,NaN,NaN,0.0,1.0,0.0,...,NaN,2019-06-30 13:09_AustinSol2019-stats.csv,2019.0,225.0,mika_c,1.0,0.0,1.000000,NaN,18
29617,wolfe_j_1507,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,...,NaN,2019-06-30 13:09_AustinSol2019-stats.csv,2019.0,225.0,wolfe_j,1.0,1.0,0.000000,NaN,18


In [7]:
df.columns

Index(['act_player_id', 'catch', 'drop', 'goalcatch', 'callahan', 'd', 'pull',
       'callahan_throw', 'throw', 'dropped_throw', 'goal', 'miscpenalty',
       'stall', 'throwaway', 'event_type', 'our_score_end_of_point', 'game_id',
       'team_id', 'datetime', 'line', 'elapsed_time_secs',
       'point_elapsed_seconds', 'hang_time_secs', 'team', 'opponent_clean',
       'opponent', 'their_score_end_of_point', 'opponent_id', 'gameover',
       'tournamemnt', 'game_id_str', 'season', 'date_id', 'act_player',
       'throws_all', 'turnover', 'pass_perc', 'catch_perc', 'ranking2019'],
      dtype='object')

In [8]:
df['game'] = df['team'] + '_' + df['opponent'] + '_'+ df['datetime'].astype('str')

NumExpr defaulting to 4 threads.


In [9]:
cols = ['game_id', 'team_id', 'opponent_id', 'date_id']
df[cols] = df[cols].apply(np.int64)

In [10]:
df.dtypes

act_player_id                object
catch                       float64
drop                        float64
goalcatch                   float64
callahan                    float64
d                           float64
pull                        float64
callahan_throw              float64
throw                       float64
dropped_throw               float64
goal                        float64
miscpenalty                 float64
stall                       float64
throwaway                   float64
event_type                   object
our_score_end_of_point      float64
game_id                       int64
team_id                       int64
datetime                     object
line                         object
elapsed_time_secs           float64
point_elapsed_seconds       float64
hang_time_secs              float64
team                         object
opponent_clean               object
opponent                     object
their_score_end_of_point    float64
opponent_id                 

In [11]:
#df['game_id'] = df['game_id'].astype(int)
#df['team_id'] = df['team_id'].astype(int)
#df['opponent_id'] = df['opponent_id'].astype(int)
#df['date_id'] = df['date_id'].astype(int)

In [12]:
tin1 = df[['team_id', 'opponent_id']].to_numpy()

In [13]:
df['game'] = ['_'.join(sorted(filter(None, x))) for x in tin1.astype('int').astype('str')]

In [14]:
df['game'].nunique()

105

In [15]:
df['game'] = df['game'] + '_' + df['date_id'].astype(str)

In [16]:
df.game.nunique()

832

In [17]:
df.game

0         22_28_0
1         22_28_0
2         22_28_0
3         22_28_0
4         22_28_0
           ...   
29614     0_5_224
29615     0_1_225
29616     0_1_225
29617     0_1_225
29618    19_5_236
Name: game, Length: 29619, dtype: object

## Groupby

In [18]:
df.columns

Index(['act_player_id', 'catch', 'drop', 'goalcatch', 'callahan', 'd', 'pull',
       'callahan_throw', 'throw', 'dropped_throw', 'goal', 'miscpenalty',
       'stall', 'throwaway', 'event_type', 'our_score_end_of_point', 'game_id',
       'team_id', 'datetime', 'line', 'elapsed_time_secs',
       'point_elapsed_seconds', 'hang_time_secs', 'team', 'opponent_clean',
       'opponent', 'their_score_end_of_point', 'opponent_id', 'gameover',
       'tournamemnt', 'game_id_str', 'season', 'date_id', 'act_player',
       'throws_all', 'turnover', 'pass_perc', 'catch_perc', 'ranking2019',
       'game'],
      dtype='object')

In [19]:
data_cols = df.columns[0:9]
data_cols

Index(['act_player_id', 'catch', 'drop', 'goalcatch', 'callahan', 'd', 'pull',
       'callahan_throw', 'throw'],
      dtype='object')

df['hm'] = df.reset_index().groupby('game')['team_id'].nunique()
df['hm'].shape

In [20]:
data_cols= {'catch', 'drop', 
             'goalcatch', 
             'callahan', 
             'd', 
             'pull','callahan_throw', 
             'throw', 
             'dropped_throw', 
             'goal', 
             'miscpenalty','stall', 'throwaway','pass_perc','catch_perc', 'throws_all', 'turnover'} 

In [21]:
text_cols = {'ranking2019', 
             'game', 
             'game_id_str', 
             'team', 
             'tournamemnt',
       'opponent_clean', 
             'act_player_id', 
             'line', 
             'gameover',
       'datetime', 
             'event_type', 
             'hang_time_secs',
       'team_id', 
             'opponent_id', 
             'game_id', 
             'opponent', 
             'date_id', 
             'season'}

In [22]:
max_cols = {'their_score_end_of_point', 
             'our_score_end_of_point'}

In [23]:
d1 = dict.fromkeys(data_cols, ['mean', 'sum', 'std'])
#d2 = dict.fromkeys(data_cols, 'std')
#d3 = dict.fromkeys(data_cols, 'sum')
d4 = dict.fromkeys(text_cols, 'last')
d_max = dict.fromkeys(max_cols, 'max')
d = {**d1, **d4, **d_max}
#df.groupby(level='A').size()

In [24]:
d

{'catch': ['mean', 'sum', 'std'],
 'goalcatch': ['mean', 'sum', 'std'],
 'pass_perc': ['mean', 'sum', 'std'],
 'drop': ['mean', 'sum', 'std'],
 'callahan_throw': ['mean', 'sum', 'std'],
 'callahan': ['mean', 'sum', 'std'],
 'throw': ['mean', 'sum', 'std'],
 'dropped_throw': ['mean', 'sum', 'std'],
 'stall': ['mean', 'sum', 'std'],
 'miscpenalty': ['mean', 'sum', 'std'],
 'turnover': ['mean', 'sum', 'std'],
 'goal': ['mean', 'sum', 'std'],
 'pull': ['mean', 'sum', 'std'],
 'throws_all': ['mean', 'sum', 'std'],
 'd': ['mean', 'sum', 'std'],
 'throwaway': ['mean', 'sum', 'std'],
 'catch_perc': ['mean', 'sum', 'std'],
 'team_id': 'last',
 'game_id_str': 'last',
 'game_id': 'last',
 'tournamemnt': 'last',
 'team': 'last',
 'opponent_id': 'last',
 'date_id': 'last',
 'line': 'last',
 'hang_time_secs': 'last',
 'gameover': 'last',
 'datetime': 'last',
 'act_player_id': 'last',
 'season': 'last',
 'opponent': 'last',
 'ranking2019': 'last',
 'game': 'last',
 'event_type': 'last',
 'opponent_cl

In [25]:
#df['game_id'] = df['game_id'].astype(int)
#df['team_id'] = df['team_id'].astype(int)
#df['opponent_id'] = df['opponent_id'].astype(int)
#df['date_id'] = df['date_id'].astype(int)


In [26]:
games = df.groupby(['game','team_id']).agg(d)

In [27]:
games.shape

(1552, 71)

In [28]:
games

catch                   goalcatch                  \
                       mean    sum        std      mean   sum       std   
game     team_id                                                          
0_14_10  14       13.157895  250.0  11.610642  1.052632  20.0  1.268143   
0_14_107 0        11.650000  233.0  10.534530  1.850000  37.0  2.230766   
         14       13.300000  266.0  12.290775  1.350000  27.0  1.980829   
0_14_118 0        12.500000  250.0   9.185228  1.550000  31.0  1.605091   
         14       12.684211  241.0   7.895517  1.000000  19.0  1.054093   
...                     ...    ...        ...       ...   ...       ...   
7_8_75   8        11.550000  231.0   9.588782  1.450000  29.0  1.538112   
7_8_9    7        15.900000  318.0  12.451590  0.800000  16.0  1.399248   
         8         9.350000  187.0   5.173566  1.500000  30.0  1.638356   
7_8_91   7        15.000000  300.0   9.049571  1.100000  22.0  1.447321   
         8        10.450000  209.0   4.260899  1.450000  29.0  1.099043   

                 pass_perc                           drop  ...    datetime  \
                      mean        sum       std      mean  ...        last   
game     team_id                                           ...               
0_14_10  14       0.919517  18.390346  0.097489  0.210526  ...  2015-05-03   
0_14_107 0        0.948879  18.028706  0.115049  0.100000  ...  2017-04-15   
         14       0.953741  19.074813  0.059557  0.250000  ...  2017-04-15   
0_14_118 0        0.948594  18.971886  0.059116  0.200000  ...  2017-05-13   
         14       0.892631  16.959996  0.112348  0.263158  ...  2017-05-13   
...                    ...        ...       ...       ...  ...         ...   
7_8_75   8        0.936228  18.724552  0.057652  0.250000  ...  2016-05-21   
7_8_9    7        0.925823  18.516450  0.083125  0.300000  ...  2015-05-02   
         8        0.941480  18.829604  0.073437  0.150000  ...  2015-05-02   
7_8_91   7        0.947674  18.953487  0.049049  0.350000  ...  2016-06-25   
         8        0.920149  18.402970  0.076695  0.150000  ...  2016-06-25   

                 act_player_id  season               opponent ranking2019  \
                          last    last                   last        last   
game     team_id                                                            
0_14_10  14         moore_j_85  2015.0          atlantahustle           0   
0_14_107 0         vickroy_733  2017.0    nashvillenightwatch          11   
         14       z_avello_732  2017.0          atlantahustle           0   
0_14_118 0         vickroy_812  2017.0    nashvillenightwatch          11   
         14         slater_809  2017.0          atlantahustle           0   
...                        ...     ...                    ...         ...   
7_8_75   8        zebart_z_501  2016.0        detroitmechanix           5   
7_8_9    7            wasdi_70  2015.0  indianapolisalleycats          21   
         8          young_l_76  2015.0        detroitmechanix           5   
7_8_91   7             wes_620  2016.0  indianapolisalleycats          21   
         8        zebart_z_623  2016.0        detroitmechanix           5   

                      game event_type         opponent_clean  \
                      last       last                   last   
game     team_id                                               
0_14_10  14        0_14_10    Offense          atlantahustle   
0_14_107 0        0_14_107    Offense    nashvillenightwatch   
         14       0_14_107    Offense          atlantahustle   
0_14_118 0        0_14_118    Offense    nashvillenightwatch   
         14       0_14_118    Offense          atlantahustle   
...                    ...        ...                    ...   
7_8_75   8          7_8_75    Offense        detroitmechanix   
7_8_9    7           7_8_9    Offense  indianapolisalleycats   
         8           7_8_9    Offense        detroitmechanix   
7_8_91   7          7_8_91    O

### Flatten multilevel columns

In [29]:
games.columns = games.columns.set_levels(games.columns.levels[1].astype(str), level=1)
games.columns = ["_".join(a) for a in games.columns.to_flat_index()]

In [30]:
games

catch_mean  catch_sum  catch_std  goalcatch_mean  \
game     team_id                                                     
0_14_10  14        13.157895      250.0  11.610642        1.052632   
0_14_107 0         11.650000      233.0  10.534530        1.850000   
         14        13.300000      266.0  12.290775        1.350000   
0_14_118 0         12.500000      250.0   9.185228        1.550000   
         14        12.684211      241.0   7.895517        1.000000   
...                      ...        ...        ...             ...   
7_8_75   8         11.550000      231.0   9.588782        1.450000   
7_8_9    7         15.900000      318.0  12.451590        0.800000   
         8          9.350000      187.0   5.173566        1.500000   
7_8_91   7         15.000000      300.0   9.049571        1.100000   
         8         10.450000      209.0   4.260899        1.450000   

                  goalcatch_sum  goalcatch_std  pass_perc_mean  pass_perc_sum  \
game     team_id                                                                
0_14_10  14                20.0       1.268143        0.919517      18.390346   
0_14_107 0                 37.0       2.230766        0.948879      18.028706   
         14                27.0       1.980829        0.953741      19.074813   
0_14_118 0                 31.0       1.605091        0.948594      18.971886   
         14                19.0       1.054093        0.892631      16.959996   
...                         ...            ...             ...            ...   
7_8_75   8                 29.0       1.538112        0.936228      18.724552   
7_8_9    7                 16.0       1.399248        0.925823      18.516450   
         8                 30.0       1.638356        0.941480      18.829604   
7_8_91   7                 22.0       1.447321        0.947674      18.953487   
         8                 29.0       1.099043        0.920149      18.402970   

                  pass_perc_std  drop_mean  ...  datetime_last  \
game     team_id                            ...                  
0_14_10  14            0.097489   0.210526  ...     2015-05-03   
0_14_107 0             0.115049   0.100000  ...     2017-04-15   
         14            0.059557   0.250000  ...     2017-04-15   
0_14_118 0             0.059116   0.200000  ...     2017-05-13   
         14            0.112348   0.263158  ...     2017-05-13   
...                         ...        ...  ...            ...   
7_8_75   8             0.057652   0.250000  ...     2016-05-21   
7_8_9    7             0.083125   0.300000  ...     2015-05-02   
         8             0.073437   0.150000  ...     2015-05-02   
7_8_91   7             0.049049   0.350000  ...     2016-06-25   
         8             0.076695   0.150000  ...     2016-06-25   

                  act_player_id_last  season_last          opponent_last  \
game     team_id                                                           
0_14_10  14               moore_j_85       2015.0          atlantahustle   
0_14_107 0               vickroy_733       2017.0    nashvillenightwatch   
         14             z_avello_732       2017.0          atlantahustle   
0_14_118 0               vickroy_812       2017.0    nashvillenightwatch   
         14               slater_809       2017.0          atlantahustle   
...                              ...          ...                    ...   
7_8_75   8              zebart_z_501       2016.0        detroitmechanix   
7_8_9    7                  wasdi_70       2015.0  indianapolisalleycats   
         8                young_l_76       2015.0        detroitmechanix   
7_8_91   7                   wes_620       2016.0  indianapolisalleycats   
         8              zebart_z_623       2016.0        detroitmechanix   

                  ranking2019_last  game_last  event_type_last  \
game     team_id                                                 
0_14_10  14                      0    0_14_10          Offense   
0_14_10

### Drop non dup rows 

In [31]:
games['game_dup'] = games.index

In [32]:
games.game_dup

game      team_id
0_14_10   14          (0_14_10, 14)
0_14_107  0           (0_14_107, 0)
          14         (0_14_107, 14)
0_14_118  0           (0_14_118, 0)
          14         (0_14_118, 14)
                          ...      
7_8_75    8             (7_8_75, 8)
7_8_9     7              (7_8_9, 7)
          8              (7_8_9, 8)
7_8_91    7             (7_8_91, 7)
          8             (7_8_91, 8)
Name: game_dup, Length: 1552, dtype: object

In [33]:
games['game_dup'] = games.reset_index(level='team_id', drop=False).index

In [34]:
games.game_dup.values

array(['0_14_10', '0_14_107', '0_14_107', ..., '7_8_9', '7_8_91',
       '7_8_91'], dtype=object)

In [35]:
games

catch_mean  catch_sum  catch_std  goalcatch_mean  \
game     team_id                                                     
0_14_10  14        13.157895      250.0  11.610642        1.052632   
0_14_107 0         11.650000      233.0  10.534530        1.850000   
         14        13.300000      266.0  12.290775        1.350000   
0_14_118 0         12.500000      250.0   9.185228        1.550000   
         14        12.684211      241.0   7.895517        1.000000   
...                      ...        ...        ...             ...   
7_8_75   8         11.550000      231.0   9.588782        1.450000   
7_8_9    7         15.900000      318.0  12.451590        0.800000   
         8          9.350000      187.0   5.173566        1.500000   
7_8_91   7         15.000000      300.0   9.049571        1.100000   
         8         10.450000      209.0   4.260899        1.450000   

                  goalcatch_sum  goalcatch_std  pass_perc_mean  pass_perc_sum  \
game     team_id                                                                
0_14_10  14                20.0       1.268143        0.919517      18.390346   
0_14_107 0                 37.0       2.230766        0.948879      18.028706   
         14                27.0       1.980829        0.953741      19.074813   
0_14_118 0                 31.0       1.605091        0.948594      18.971886   
         14                19.0       1.054093        0.892631      16.959996   
...                         ...            ...             ...            ...   
7_8_75   8                 29.0       1.538112        0.936228      18.724552   
7_8_9    7                 16.0       1.399248        0.925823      18.516450   
         8                 30.0       1.638356        0.941480      18.829604   
7_8_91   7                 22.0       1.447321        0.947674      18.953487   
         8                 29.0       1.099043        0.920149      18.402970   

                  pass_perc_std  drop_mean  ...  act_player_id_last  \
game     team_id                            ...                       
0_14_10  14            0.097489   0.210526  ...          moore_j_85   
0_14_107 0             0.115049   0.100000  ...         vickroy_733   
         14            0.059557   0.250000  ...        z_avello_732   
0_14_118 0             0.059116   0.200000  ...         vickroy_812   
         14            0.112348   0.263158  ...          slater_809   
...                         ...        ...  ...                 ...   
7_8_75   8             0.057652   0.250000  ...        zebart_z_501   
7_8_9    7             0.083125   0.300000  ...            wasdi_70   
         8             0.073437   0.150000  ...          young_l_76   
7_8_91   7             0.049049   0.350000  ...             wes_620   
         8             0.076695   0.150000  ...        zebart_z_623   

                  season_last          opponent_last  ranking2019_last  \
game     team_id                                                         
0_14_10  14            2015.0          atlantahustle                 0   
0_14_107 0             2017.0    nashvillenightwatch                11   
         14            2017.0          atlantahustle                 0   
0_14_118 0             2017.0    nashvillenightwatch                11   
         14            2017.0          atlantahustle                 0   
...                       ...                    ...               ...   
7_8_75   8             2016.0        detroitmechanix                 5   
7_8_9    7             2015.0  indianapolisalleycats                21   
         8             2015.0        detroitmechanix                 5   
7_8_91   7             2016.0  indianapolisalleycats                21   
         8             2016.0        detroitmechanix                 5   

                  game_last  event_type_last    opponent_clean_last  \
game     team_id                                                      
0_14_10  14         0_14

In [36]:
dups = games['game_dup'].duplicated(keep=False).values
games['game_dups_bool'] = dups
games.game_dups_bool.value_counts()

True     1440
False     112
Name: game_dups_bool, dtype: int64

In [37]:
games.loc[games.game_dups_bool ==False].shape
# Drop rows when game_id_dups equals False (get rid of games with only one-sided data)


(112, 73)

In [38]:
games = games.loc[games.game_dups_bool, :]

In [39]:
games.shape

(1440, 73)

In [40]:
games

catch_mean  catch_sum  catch_std  goalcatch_mean  \
game     team_id                                                     
0_14_107 0         11.650000      233.0  10.534530            1.85   
         14        13.300000      266.0  12.290775            1.35   
0_14_118 0         12.500000      250.0   9.185228            1.55   
         14        12.684211      241.0   7.895517            1.00   
0_14_140 0         13.850000      277.0  13.437085            1.35   
...                      ...        ...        ...             ...   
7_8_75   8         11.550000      231.0   9.588782            1.45   
7_8_9    7         15.900000      318.0  12.451590            0.80   
         8          9.350000      187.0   5.173566            1.50   
7_8_91   7         15.000000      300.0   9.049571            1.10   
         8         10.450000      209.0   4.260899            1.45   

                  goalcatch_sum  goalcatch_std  pass_perc_mean  pass_perc_sum  \
game     team_id                                                                
0_14_107 0                 37.0       2.230766        0.948879      18.028706   
         14                27.0       1.980829        0.953741      19.074813   
0_14_118 0                 31.0       1.605091        0.948594      18.971886   
         14                19.0       1.054093        0.892631      16.959996   
0_14_140 0                 27.0       1.531253        0.933650      17.739348   
...                         ...            ...             ...            ...   
7_8_75   8                 29.0       1.538112        0.936228      18.724552   
7_8_9    7                 16.0       1.399248        0.925823      18.516450   
         8                 30.0       1.638356        0.941480      18.829604   
7_8_91   7                 22.0       1.447321        0.947674      18.953487   
         8                 29.0       1.099043        0.920149      18.402970   

                  pass_perc_std  drop_mean  ...  season_last  \
game     team_id                            ...                
0_14_107 0             0.115049   0.100000  ...       2017.0   
         14            0.059557   0.250000  ...       2017.0   
0_14_118 0             0.059116   0.200000  ...       2017.0   
         14            0.112348   0.263158  ...       2017.0   
0_14_140 0             0.080997   0.050000  ...       2017.0   
...                         ...        ...  ...          ...   
7_8_75   8             0.057652   0.250000  ...       2016.0   
7_8_9    7             0.083125   0.300000  ...       2015.0   
         8             0.073437   0.150000  ...       2015.0   
7_8_91   7             0.049049   0.350000  ...       2016.0   
         8             0.076695   0.150000  ...       2016.0   

                          opponent_last  ranking2019_last  game_last  \
game     team_id                                                       
0_14_107 0          nashvillenightwatch                11   0_14_107   
         14               atlantahustle                 0   0_14_107   
0_14_118 0          nashvillenightwatch                11   0_14_118   
         14               atlantahustle                 0   0_14_118   
0_14_140 0          nashvillenightwatch                11   0_14_140   
...                                 ...               ...        ...   
7_8_75   8              detroitmechanix                 5     7_8_75   
7_8_9    7        indianapolisalleycats                21      7_8_9   
         8              detroitmechanix                 5      7_8_9   
7_8_91   7        indianapolisalleycats                21     7_8_91   
         8              detroitmechanix                 5     7_8_91   

                  event_type_last    opponent_clean_last  \
game     team_id                                           
0_14_107 0                Offense    nashvillenightwatch   
         14               Offense          atlantahustle   
0_14_118 0                Offense    nashvi

### Create t1 and t2 column

In [41]:
#games.index(level='team_id')

In [42]:
#games.index.levels[1] = games['team_t']

In [43]:
games.index = games.index.set_levels(games.index.levels[1].astype(str), level=1)

In [44]:
len(games.index.levels[1])

29

In [45]:
games.shape

(1440, 73)

In [46]:
#games.index = games.index.set_levels(games['team_t'], level=1)

In [47]:
#df.index = df.index.set_levels(df.index.levels[0].str.replace(' ', ''), level=0)

In [48]:
games

catch_mean  catch_sum  catch_std  goalcatch_mean  \
game     team_id                                                     
0_14_107 0         11.650000      233.0  10.534530            1.85   
         14        13.300000      266.0  12.290775            1.35   
0_14_118 0         12.500000      250.0   9.185228            1.55   
         14        12.684211      241.0   7.895517            1.00   
0_14_140 0         13.850000      277.0  13.437085            1.35   
...                      ...        ...        ...             ...   
7_8_75   8         11.550000      231.0   9.588782            1.45   
7_8_9    7         15.900000      318.0  12.451590            0.80   
         8          9.350000      187.0   5.173566            1.50   
7_8_91   7         15.000000      300.0   9.049571            1.10   
         8         10.450000      209.0   4.260899            1.45   

                  goalcatch_sum  goalcatch_std  pass_perc_mean  pass_perc_sum  \
game     team_id                                                                
0_14_107 0                 37.0       2.230766        0.948879      18.028706   
         14                27.0       1.980829        0.953741      19.074813   
0_14_118 0                 31.0       1.605091        0.948594      18.971886   
         14                19.0       1.054093        0.892631      16.959996   
0_14_140 0                 27.0       1.531253        0.933650      17.739348   
...                         ...            ...             ...            ...   
7_8_75   8                 29.0       1.538112        0.936228      18.724552   
7_8_9    7                 16.0       1.399248        0.925823      18.516450   
         8                 30.0       1.638356        0.941480      18.829604   
7_8_91   7                 22.0       1.447321        0.947674      18.953487   
         8                 29.0       1.099043        0.920149      18.402970   

                  pass_perc_std  drop_mean  ...  season_last  \
game     team_id                            ...                
0_14_107 0             0.115049   0.100000  ...       2017.0   
         14            0.059557   0.250000  ...       2017.0   
0_14_118 0             0.059116   0.200000  ...       2017.0   
         14            0.112348   0.263158  ...       2017.0   
0_14_140 0             0.080997   0.050000  ...       2017.0   
...                         ...        ...  ...          ...   
7_8_75   8             0.057652   0.250000  ...       2016.0   
7_8_9    7             0.083125   0.300000  ...       2015.0   
         8             0.073437   0.150000  ...       2015.0   
7_8_91   7             0.049049   0.350000  ...       2016.0   
         8             0.076695   0.150000  ...       2016.0   

                          opponent_last  ranking2019_last  game_last  \
game     team_id                                                       
0_14_107 0          nashvillenightwatch                11   0_14_107   
         14               atlantahustle                 0   0_14_107   
0_14_118 0          nashvillenightwatch                11   0_14_118   
         14               atlantahustle                 0   0_14_118   
0_14_140 0          nashvillenightwatch                11   0_14_140   
...                                 ...               ...        ...   
7_8_75   8              detroitmechanix                 5     7_8_75   
7_8_9    7        indianapolisalleycats                21      7_8_9   
         8              detroitmechanix                 5      7_8_9   
7_8_91   7        indianapolisalleycats                21     7_8_91   
         8              detroitmechanix                 5     7_8_91   

                  event_type_last    opponent_clean_last  \
game     team_id                                           
0_14_107 0                Offense    nashvillenightwatch   
         14               Offense          atlantahustle   
0_14_118 0                Offense    nashvi

In [49]:
games['team_t'] = games['team_id_last']

<ipython-input-49-43920952e385>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  games['team_t'] = games['team_id_last']


In [50]:
games.team_t.iloc[0::2] = 't1'
games.team_t.iloc[1::2] = 't2'

/Users/ilonanietosvaara/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
/Users/ilonanietosvaara/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:692: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value, self.name)


In [51]:
games

catch_mean  catch_sum  catch_std  goalcatch_mean  \
game     team_id                                                     
0_14_107 0         11.650000      233.0  10.534530            1.85   
         14        13.300000      266.0  12.290775            1.35   
0_14_118 0         12.500000      250.0   9.185228            1.55   
         14        12.684211      241.0   7.895517            1.00   
0_14_140 0         13.850000      277.0  13.437085            1.35   
...                      ...        ...        ...             ...   
7_8_75   8         11.550000      231.0   9.588782            1.45   
7_8_9    7         15.900000      318.0  12.451590            0.80   
         8          9.350000      187.0   5.173566            1.50   
7_8_91   7         15.000000      300.0   9.049571            1.10   
         8         10.450000      209.0   4.260899            1.45   

                  goalcatch_sum  goalcatch_std  pass_perc_mean  pass_perc_sum  \
game     team_id                                                                
0_14_107 0                 37.0       2.230766        0.948879      18.028706   
         14                27.0       1.980829        0.953741      19.074813   
0_14_118 0                 31.0       1.605091        0.948594      18.971886   
         14                19.0       1.054093        0.892631      16.959996   
0_14_140 0                 27.0       1.531253        0.933650      17.739348   
...                         ...            ...             ...            ...   
7_8_75   8                 29.0       1.538112        0.936228      18.724552   
7_8_9    7                 16.0       1.399248        0.925823      18.516450   
         8                 30.0       1.638356        0.941480      18.829604   
7_8_91   7                 22.0       1.447321        0.947674      18.953487   
         8                 29.0       1.099043        0.920149      18.402970   

                  pass_perc_std  drop_mean  ...          opponent_last  \
game     team_id                            ...                          
0_14_107 0             0.115049   0.100000  ...    nashvillenightwatch   
         14            0.059557   0.250000  ...          atlantahustle   
0_14_118 0             0.059116   0.200000  ...    nashvillenightwatch   
         14            0.112348   0.263158  ...          atlantahustle   
0_14_140 0             0.080997   0.050000  ...    nashvillenightwatch   
...                         ...        ...  ...                    ...   
7_8_75   8             0.057652   0.250000  ...        detroitmechanix   
7_8_9    7             0.083125   0.300000  ...  indianapolisalleycats   
         8             0.073437   0.150000  ...        detroitmechanix   
7_8_91   7             0.049049   0.350000  ...  indianapolisalleycats   
         8             0.076695   0.150000  ...        detroitmechanix   

                  ranking2019_last  game_last  event_type_last  \
game     team_id                                                 
0_14_107 0                      11   0_14_107          Offense   
         14                      0   0_14_107          Offense   
0_14_118 0                      11   0_14_118          Offense   
         14                      0   0_14_118          Offense   
0_14_140 0                      11   0_14_140          Offense   
...                            ...        ...              ...   
7_8_75   8                       5     7_8_75          Offense   
7_8_9    7                      21      7_8_9          Offense   
         8                       5      7_8_9          Offense   
7_8_91   7                      21     7_8_91          Offense   
         8                       5     7_8_91          Offense   

                    opponent_clean_last  our_score_end_of_point_max  \
game     team_id                                                      
0_14_107 0          nashvillenightwatch                        37.0   
         14       

In [52]:
games.team_t

game      team_id
0_14_107  0          t1
          14         t2
0_14_118  0          t1
          14         t2
0_14_140  0          t1
                     ..
7_8_75    8          t2
7_8_9     7          t1
          8          t2
7_8_91    7          t1
          8          t2
Name: team_t, Length: 1440, dtype: object

In [53]:
len(games.index.levels[1])

29

In [54]:
#games['team_t'] = np.nan
#games.iloc[0::2, 0] = 't1'
#games.iloc[1::2, 0] = 't2'

In [55]:
len(games.index.levels[0])

832

In [56]:
#games.index = games.index.set_levels(games.index.levels[1] == games['team_t'], level=1)

In [57]:
games

catch_mean  catch_sum  catch_std  goalcatch_mean  \
game     team_id                                                     
0_14_107 0         11.650000      233.0  10.534530            1.85   
         14        13.300000      266.0  12.290775            1.35   
0_14_118 0         12.500000      250.0   9.185228            1.55   
         14        12.684211      241.0   7.895517            1.00   
0_14_140 0         13.850000      277.0  13.437085            1.35   
...                      ...        ...        ...             ...   
7_8_75   8         11.550000      231.0   9.588782            1.45   
7_8_9    7         15.900000      318.0  12.451590            0.80   
         8          9.350000      187.0   5.173566            1.50   
7_8_91   7         15.000000      300.0   9.049571            1.10   
         8         10.450000      209.0   4.260899            1.45   

                  goalcatch_sum  goalcatch_std  pass_perc_mean  pass_perc_sum  \
game     team_id                                                                
0_14_107 0                 37.0       2.230766        0.948879      18.028706   
         14                27.0       1.980829        0.953741      19.074813   
0_14_118 0                 31.0       1.605091        0.948594      18.971886   
         14                19.0       1.054093        0.892631      16.959996   
0_14_140 0                 27.0       1.531253        0.933650      17.739348   
...                         ...            ...             ...            ...   
7_8_75   8                 29.0       1.538112        0.936228      18.724552   
7_8_9    7                 16.0       1.399248        0.925823      18.516450   
         8                 30.0       1.638356        0.941480      18.829604   
7_8_91   7                 22.0       1.447321        0.947674      18.953487   
         8                 29.0       1.099043        0.920149      18.402970   

                  pass_perc_std  drop_mean  ...          opponent_last  \
game     team_id                            ...                          
0_14_107 0             0.115049   0.100000  ...    nashvillenightwatch   
         14            0.059557   0.250000  ...          atlantahustle   
0_14_118 0             0.059116   0.200000  ...    nashvillenightwatch   
         14            0.112348   0.263158  ...          atlantahustle   
0_14_140 0             0.080997   0.050000  ...    nashvillenightwatch   
...                         ...        ...  ...                    ...   
7_8_75   8             0.057652   0.250000  ...        detroitmechanix   
7_8_9    7             0.083125   0.300000  ...  indianapolisalleycats   
         8             0.073437   0.150000  ...        detroitmechanix   
7_8_91   7             0.049049   0.350000  ...  indianapolisalleycats   
         8             0.076695   0.150000  ...        detroitmechanix   

                  ranking2019_last  game_last  event_type_last  \
game     team_id                                                 
0_14_107 0                      11   0_14_107          Offense   
         14                      0   0_14_107          Offense   
0_14_118 0                      11   0_14_118          Offense   
         14                      0   0_14_118          Offense   
0_14_140 0                      11   0_14_140          Offense   
...                            ...        ...              ...   
7_8_75   8                       5     7_8_75          Offense   
7_8_9    7                      21      7_8_9          Offense   
         8                       5      7_8_9          Offense   
7_8_91   7                      21     7_8_91          Offense   
         8                       5     7_8_91          Offense   

                    opponent_clean_last  our_score_end_of_point_max  \
game     team_id                                                      
0_14_107 0          nashvillenightwatch                        37.0   
         14       

In [58]:
#games.index = games.index.set_levels(games.index.levels[1] == games['team', level=1)

## New dataframe reset_games with no multilevel index

In [59]:
reset_games = games.reset_index(drop=False)

In [60]:
reset_games

,game,team_id,catch_mean,catch_sum,catch_std,goalcatch_mean,goalcatch_sum,goalcatch_std,pass_perc_mean,pass_perc_sum,...,opponent_last,ranking2019_last,game_last,event_type_last,opponent_clean_last,our_score_end_of_point_max,their_score_end_of_point_max,game_dup,game_dups_bool,team_t
0,0_14_107,0,11.650000,233.0,10.534530,1.85,37.0,2.230766,0.948879,18.028706,...,nashvillenightwatch,11,0_14_107,Offense,nashvillenightwatch,37.0,27.0,0_14_107,True,t1
1,0_14_107,14,13.300000,266.0,12.290775,1.35,27.0,1.980829,0.953741,19.074813,...,atlantahustle,0,0_14_107,Offense,atlantahustle,27.0,37.0,0_14_107,True,t2
2,0_14_118,0,12.500000,250.0,9.185228,1.55,31.0,1.605091,0.948594,18.971886,...,nashvillenightwatch,11,0_14_118,Offense,nashvillenightwatch,31.0,18.0,0_14_118,True,t1
3,0_14_118,14,12.684211,241.0,7.895517,1.00,19.0,1.054093,0.892631,16.959996,...,atlantahustle,0,0_14_118,Offense,atlantahustle,19.0,31.0,0_14_118,True,t2
4,0_14_140,0,13.850000,277.0,13.437085,1.35,27.0,1.531253,0.933650,17.739348,...,nashvillenightwatch,11,0_14_140,Offense,nashvillenightwatch,27.0,22.0,0_14_140,True,t1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1435,7_8_75,8,11.550000,231.0,9.588782,1.45,29.0,1.538112,0.936228,18.724552,...,detroitmechanix,5,7_8_75,Offense,detroitmechanix,29.0,24.0,7_8_75,True,t2
1436,7_8_9,7,15.900000,318.0,12.451590,0.80,16.0,1.399248,0.925823,18.516450,...,indianapolisalleycats,21,7_8_9,Offense,indianapolisalleycats,16.0,30.0,7_8_9,True,t1
1437,7_8_9,8,9.350000,187.0,5.173566,1.50,30.0,1.638356,0.941480,18.829604,...,detroitmechanix,5,7_8_9,Offense,detroitmechanix,30.0,16.0,7_8_9,True,t2
1438,7_8_91,7,15.000000,300.0,9.049571,1.10,22.0,1.447321,0.947674,18.953487,...,indianapolisalleycats,21,7_8_91,Offense,indianapolisalleycats,22.0,29.0,7_8_91,True,t1


In [61]:
reset_games.set_index(['game', 'team_t'], drop=False, inplace=True)

In [62]:
reset_games

game team_id  catch_mean  catch_sum  catch_std  \
game     team_t                                                       
0_14_107 t1      0_14_107       0   11.650000      233.0  10.534530   
         t2      0_14_107      14   13.300000      266.0  12.290775   
0_14_118 t1      0_14_118       0   12.500000      250.0   9.185228   
         t2      0_14_118      14   12.684211      241.0   7.895517   
0_14_140 t1      0_14_140       0   13.850000      277.0  13.437085   
...                   ...     ...         ...        ...        ...   
7_8_75   t2        7_8_75       8   11.550000      231.0   9.588782   
7_8_9    t1         7_8_9       7   15.900000      318.0  12.451590   
         t2         7_8_9       8    9.350000      187.0   5.173566   
7_8_91   t1        7_8_91       7   15.000000      300.0   9.049571   
         t2        7_8_91       8   10.450000      209.0   4.260899   

                 goalcatch_mean  goalcatch_sum  goalcatch_std  pass_perc_mean  \
game     team_t                                                                 
0_14_107 t1                1.85           37.0       2.230766        0.948879   
         t2                1.35           27.0       1.980829        0.953741   
0_14_118 t1                1.55           31.0       1.605091        0.948594   
         t2                1.00           19.0       1.054093        0.892631   
0_14_140 t1                1.35           27.0       1.531253        0.933650   
...                         ...            ...            ...             ...   
7_8_75   t2                1.45           29.0       1.538112        0.936228   
7_8_9    t1                0.80           16.0       1.399248        0.925823   
         t2                1.50           30.0       1.638356        0.941480   
7_8_91   t1                1.10           22.0       1.447321        0.947674   
         t2                1.45           29.0       1.099043        0.920149   

                 pass_perc_sum  ...          opponent_last  ranking2019_last  \
game     team_t                 ...                                            
0_14_107 t1          18.028706  ...    nashvillenightwatch                11   
         t2          19.074813  ...          atlantahustle                 0   
0_14_118 t1          18.971886  ...    nashvillenightwatch                11   
         t2          16.959996  ...          atlantahustle                 0   
0_14_140 t1          17.739348  ...    nashvillenightwatch                11   
...                        ...  ...                    ...               ...   
7_8_75   t2          18.724552  ...        detroitmechanix                 5   
7_8_9    t1          18.516450  ...  indianapolisalleycats                21   
         t2          18.829604  ...        detroitmechanix                 5   
7_8_91   t1          18.953487  ...  indianapolisalleycats                21   
         t2          18.402970  ...        detroitmechanix                 5   

                 game_last  event_type_last    opponent_clean_last  \
game     team_t                                                      
0_14_107 t1       0_14_107          Offense    nashvillenightwatch   
         t2       0_14_107          Offense          atlantahustle   
0_14_118 t1       0_14_118          Offense    nashvillenightwatch   
         t2       0_14_118          Offense          atlantahustle   
0_14_140 t1       0_14_140          Offense    nashvillenightwatch   
...                    ...              ...                    ...   
7_8_75   t2         7_8_75          Offense        detroitmechanix   
7_8_9    t1          7_8_9          Offense  indianapolisalleycats   
         t2          7_8_9          Offense        detroitmechanix   
7_8_91   t1         7_8_91          Offense  indianapolisalleycats   
         t2         7_8_91          Offense        detroitmechanix   

                 our_score_end_of_point_max  their_score_end_of_point_max  \
game     team_

In [63]:
unstacked = reset_games.unstack()

In [64]:
unstacked

game           team_id     catch_mean            catch_sum  \
team_t          t1        t2      t1  t2         t1         t2        t1   
game                                                                       
0_14_107  0_14_107  0_14_107       0  14  11.650000  13.300000     233.0   
0_14_118  0_14_118  0_14_118       0  14  12.500000  12.684211     250.0   
0_14_140  0_14_140  0_14_140       0  14  13.850000   0.100000     277.0   
0_14_157  0_14_157  0_14_157       0  14  14.100000  13.700000     282.0   
0_14_176  0_14_176  0_14_176       0  14  12.000000  13.600000     240.0   
...            ...       ...     ...  ..        ...        ...       ...   
7_8_203    7_8_203   7_8_203       7   8  14.866667  10.850000     223.0   
7_8_28      7_8_28    7_8_28       7   8  16.588235  10.800000     282.0   
7_8_75      7_8_75    7_8_75       7   8  14.882353  11.550000     253.0   
7_8_9        7_8_9     7_8_9       7   8  15.900000   9.350000     318.0   
7_8_91      7_8_91    7_8_91       7   8  15.000000  10.450000     300.0   

                 catch_std             ... our_score_end_of_point_max        \
team_t       t2         t1         t2  ...                         t1    t2   
game                                   ...                                    
0_14_107  266.0  10.534530  12.290775  ...                       37.0  27.0   
0_14_118  241.0   9.185228   7.895517  ...                       31.0  19.0   
0_14_140    1.0  13.437085   0.316228  ...                       27.0  22.0   
0_14_157  274.0  10.808866  16.921180  ...                       18.0  16.0   
0_14_176  272.0   6.521220  17.578096  ...                       24.0  21.0   
...         ...        ...        ...  ...                        ...   ...   
7_8_203   217.0  19.386544  11.672167  ...                       16.0  19.0   
7_8_28    216.0  16.431900   9.860074  ...                       21.0  29.0   
7_8_75    231.0  13.564118   9.588782  ...                       24.0  29.0   
7_8_9     187.0  12.451590   5.173566  ...                       16.0  30.0   
7_8_91    209.0   9.049571   4.260899  ...                       22.0  29.0   

         their_score_end_of_point_max        game_dup            \
team_t                             t1    t2        t1        t2   
game                                                              
0_14_107                         27.0  37.0  0_14_107  0_14_107   
0_14_118                         18.0  31.0  0_14_118  0_14_118   
0_14_140                         22.0  27.0  0_14_140  0_14_140   
0_14_157                         17.0  15.0  0_14_157  0_14_157   
0_14_176                         20.0  24.0  0_14_176  0_14_176   
...                               ...   ...       ...       ...   
7_8_203                          19.0  16.0   7_8_203   7_8_203   
7_8_28                           29.0  21.0    7_8_28    7_8_28   
7_8_75                           29.0  24.0    7_8_75    7_8_75   
7_8_9                            30.0  16.0     7_8_9     7_8_9   
7_8_91                           29.0  22.0    7_8_91    7_8_91   

         game_dups_bool       team_t      
team_t               t1    t2     t1  t2  
game                                      
0_14_107           True  True     t1  t2  
0_14_118           True  True     t1  t2  
0_14_140           True  True     t1  t2  
0_14_157           True  True     t1  t2  
0_14_176           True  True     t1  t2  
...                 ...   ...    ...  ..  
7_8_203            True  True     t1  t2  
7_8_28             True  True     t1  t2  
7_8_75             True  True     t1  t2  
7_8_9              True  True     t1  t2  
7_8_91             True  True     t1  t2  

[720 rows x 152 columns]

In [65]:
unstacked.columns = unstacked.columns.set_levels(unstacked.columns.levels[1].astype(str), level=1)
unstacked.columns = ["_".join(a) for a in unstacked.columns.to_flat_index()]

In [66]:
unstacked

,game_t1,game_t2,team_id_t1,team_id_t2,catch_mean_t1,catch_mean_t2,catch_sum_t1,catch_sum_t2,catch_std_t1,catch_std_t2,...,our_score_end_of_point_max_t1,our_score_end_of_point_max_t2,their_score_end_of_point_max_t1,their_score_end_of_point_max_t2,game_dup_t1,game_dup_t2,game_dups_bool_t1,game_dups_bool_t2,team_t_t1,team_t_t2
game,,,,,,,,,,,,,,,,,,,,,
0_14_107,0_14_107,0_14_107,0,14,11.650000,13.300000,233.0,266.0,10.534530,12.290775,...,37.0,27.0,27.0,37.0,0_14_107,0_14_107,True,True,t1,t2
0_14_118,0_14_118,0_14_118,0,14,12.500000,12.684211,250.0,241.0,9.185228,7.895517,...,31.0,19.0,18.0,31.0,0_14_118,0_14_118,True,True,t1,t2
0_14_140,0_14_140,0_14_140,0,14,13.850000,0.100000,277.0,1.0,13.437085,0.316228,...,27.0,22.0,22.0,27.0,0_14_140,0_14_140,True,True,t1,t2
0_14_157,0_14_157,0_14_157,0,14,14.100000,13.700000,282.0,274.0,10.808866,16.921180,...,18.0,16.0,17.0,15.0,0_14_157,0_14_157,True,True,t1,t2
0_14_176,0_14_176,0_14_176,0,14,12.000000,13.600000,240.0,272.0,6.521220,17.578096,...,24.0,21.0,20.0,24.0,0_14_176,0_14_176,True,True,t1,t2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7_8_203,7_8_203,7_8_203,7,8,14.866667,10.850000,223.0,217.0,19.386544,11.672167,...,16.0,19.0,19.0,16.0,7_8_203,7_8_203,True,True,t1,t2
7_8_28,7_8_28,7_8_28,7,8,16.588235,10.800000,282.0,216.0,16.431900,9.860074,...,21.0,29.0,29.0,21.0,7_8_28,7_8_28,True,True,t1,t2
7_8_75,7_8_75,7_8_75,7,8,14.882353,11.550000,253.0,231.0,13.564118,9.588782,...,24.0,29.0,29.0,24.0,7_8_75,7_8_75,True,True,t1,t2


In [67]:
unstacked.columns[100:150]

Index(['catch_perc_mean_t1', 'catch_perc_mean_t2', 'catch_perc_sum_t1',
       'catch_perc_sum_t2', 'catch_perc_std_t1', 'catch_perc_std_t2',
       'team_id_last_t1', 'team_id_last_t2', 'game_id_str_last_t1',
       'game_id_str_last_t2', 'game_id_last_t1', 'game_id_last_t2',
       'tournamemnt_last_t1', 'tournamemnt_last_t2', 'team_last_t1',
       'team_last_t2', 'opponent_id_last_t1', 'opponent_id_last_t2',
       'date_id_last_t1', 'date_id_last_t2', 'line_last_t1', 'line_last_t2',
       'hang_time_secs_last_t1', 'hang_time_secs_last_t2', 'gameover_last_t1',
       'gameover_last_t2', 'datetime_last_t1', 'datetime_last_t2',
       'act_player_id_last_t1', 'act_player_id_last_t2', 'season_last_t1',
       'season_last_t2', 'opponent_last_t1', 'opponent_last_t2',
       'ranking2019_last_t1', 'ranking2019_last_t2', 'game_last_t1',
       'game_last_t2', 'event_type_last_t1', 'event_type_last_t2',
       'opponent_clean_last_t1', 'opponent_clean_last_t2',
       'our_score_end_of_p

In [68]:
unstacked[['team_id_last_t1', 'team_id_last_t2','our_score_end_of_point_max_t1','their_score_end_of_point_max_t2',
'our_score_end_of_point_max_t2',
'their_score_end_of_point_max_t1'
]]

,team_id_last_t1,team_id_last_t2,our_score_end_of_point_max_t1,their_score_end_of_point_max_t2,our_score_end_of_point_max_t2,their_score_end_of_point_max_t1
game,,,,,,
0_14_107,0,14,37.0,37.0,27.0,27.0
0_14_118,0,14,31.0,31.0,19.0,18.0
0_14_140,0,14,27.0,27.0,22.0,22.0
0_14_157,0,14,18.0,15.0,16.0,17.0
0_14_176,0,14,24.0,24.0,21.0,20.0
...,...,...,...,...,...,...
7_8_203,7,8,16.0,16.0,19.0,19.0
7_8_28,7,8,21.0,21.0,29.0,29.0
7_8_75,7,8,24.0,24.0,29.0,29.0


In [69]:
# Create win column (this is the label to predict)
unstacked['win_t1'] = (unstacked['our_score_end_of_point_max_t1'] > unstacked['their_score_end_of_point_max_t1']).astype(int)
unstacked.win_t1.value_counts()

0    400
1    320
Name: win_t1, dtype: int64

### Save data for modelling

In [70]:
unstacked.columns[-20:]

Index(['opponent_last_t2', 'ranking2019_last_t1', 'ranking2019_last_t2',
       'game_last_t1', 'game_last_t2', 'event_type_last_t1',
       'event_type_last_t2', 'opponent_clean_last_t1',
       'opponent_clean_last_t2', 'our_score_end_of_point_max_t1',
       'our_score_end_of_point_max_t2', 'their_score_end_of_point_max_t1',
       'their_score_end_of_point_max_t2', 'game_dup_t1', 'game_dup_t2',
       'game_dups_bool_t1', 'game_dups_bool_t2', 'team_t_t1', 'team_t_t2',
       'win_t1'],
      dtype='object')

In [71]:
unstacked.drop(['game_dup_t1', 'game_dup_t2',
       'game_dups_bool_t1', 'game_dups_bool_t2', 'team_t_t1', 'team_t_t2','gameover_last_t1', 'gameover_last_t2'], axis=1, inplace=True)

In [72]:
unstacked

,game_t1,game_t2,team_id_t1,team_id_t2,catch_mean_t1,catch_mean_t2,catch_sum_t1,catch_sum_t2,catch_std_t1,catch_std_t2,...,game_last_t2,event_type_last_t1,event_type_last_t2,opponent_clean_last_t1,opponent_clean_last_t2,our_score_end_of_point_max_t1,our_score_end_of_point_max_t2,their_score_end_of_point_max_t1,their_score_end_of_point_max_t2,win_t1
game,,,,,,,,,,,,,,,,,,,,,
0_14_107,0_14_107,0_14_107,0,14,11.650000,13.300000,233.0,266.0,10.534530,12.290775,...,0_14_107,Offense,Offense,nashvillenightwatch,atlantahustle,37.0,27.0,27.0,37.0,1
0_14_118,0_14_118,0_14_118,0,14,12.500000,12.684211,250.0,241.0,9.185228,7.895517,...,0_14_118,Offense,Offense,nashvillenightwatch,atlantahustle,31.0,19.0,18.0,31.0,1
0_14_140,0_14_140,0_14_140,0,14,13.850000,0.100000,277.0,1.0,13.437085,0.316228,...,0_14_140,Offense,Offense,nashvillenightwatch,atlantahustle,27.0,22.0,22.0,27.0,1
0_14_157,0_14_157,0_14_157,0,14,14.100000,13.700000,282.0,274.0,10.808866,16.921180,...,0_14_157,Offense,Offense,nashvillenightwatch,atlantahustle,18.0,16.0,17.0,15.0,1
0_14_176,0_14_176,0_14_176,0,14,12.000000,13.600000,240.0,272.0,6.521220,17.578096,...,0_14_176,Offense,Offense,nashvillenightwatch,atlantahustle,24.0,21.0,20.0,24.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7_8_203,7_8_203,7_8_203,7,8,14.866667,10.850000,223.0,217.0,19.386544,11.672167,...,7_8_203,Offense,Offense,indianapolisalleycats,detroitmechanix,16.0,19.0,19.0,16.0,0
7_8_28,7_8_28,7_8_28,7,8,16.588235,10.800000,282.0,216.0,16.431900,9.860074,...,7_8_28,Offense,Offense,indianapolisalleycats,detroitmechanix,21.0,29.0,29.0,21.0,0
7_8_75,7_8_75,7_8_75,7,8,14.882353,11.550000,253.0,231.0,13.564118,9.588782,...,7_8_75,Offense,Offense,indianapolisalleycats,detroitmechanix,24.0,29.0,29.0,24.0,0


In [73]:
unstacked.to_csv('model.csv')

In [74]:
df = pd.read_csv('model.csv', index_col = 'game')

In [75]:
df

,game_t1,game_t2,team_id_t1,team_id_t2,catch_mean_t1,catch_mean_t2,catch_sum_t1,catch_sum_t2,catch_std_t1,catch_std_t2,...,game_last_t2,event_type_last_t1,event_type_last_t2,opponent_clean_last_t1,opponent_clean_last_t2,our_score_end_of_point_max_t1,our_score_end_of_point_max_t2,their_score_end_of_point_max_t1,their_score_end_of_point_max_t2,win_t1
game,,,,,,,,,,,,,,,,,,,,,
0_14_107,0_14_107,0_14_107,0,14,11.650000,13.300000,233.0,266.0,10.534530,12.290775,...,0_14_107,Offense,Offense,nashvillenightwatch,atlantahustle,37.0,27.0,27.0,37.0,1
0_14_118,0_14_118,0_14_118,0,14,12.500000,12.684211,250.0,241.0,9.185228,7.895517,...,0_14_118,Offense,Offense,nashvillenightwatch,atlantahustle,31.0,19.0,18.0,31.0,1
0_14_140,0_14_140,0_14_140,0,14,13.850000,0.100000,277.0,1.0,13.437085,0.316228,...,0_14_140,Offense,Offense,nashvillenightwatch,atlantahustle,27.0,22.0,22.0,27.0,1
0_14_157,0_14_157,0_14_157,0,14,14.100000,13.700000,282.0,274.0,10.808866,16.921180,...,0_14_157,Offense,Offense,nashvillenightwatch,atlantahustle,18.0,16.0,17.0,15.0,1
0_14_176,0_14_176,0_14_176,0,14,12.000000,13.600000,240.0,272.0,6.521220,17.578096,...,0_14_176,Offense,Offense,nashvillenightwatch,atlantahustle,24.0,21.0,20.0,24.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7_8_203,7_8_203,7_8_203,7,8,14.866667,10.850000,223.0,217.0,19.386544,11.672167,...,7_8_203,Offense,Offense,indianapolisalleycats,detroitmechanix,16.0,19.0,19.0,16.0,0
7_8_28,7_8_28,7_8_28,7,8,16.588235,10.800000,282.0,216.0,16.431900,9.860074,...,7_8_28,Offense,Offense,indianapolisalleycats,detroitmechanix,21.0,29.0,29.0,21.0,0
7_8_75,7_8_75,7_8_75,7,8,14.882353,11.550000,253.0,231.0,13.564118,9.588782,...,7_8_75,Offense,Offense,indianapolisalleycats,detroitmechanix,24.0,29.0,29.0,24.0,0


In [76]:
df.columns

Index(['game_t1', 'game_t2', 'team_id_t1', 'team_id_t2', 'catch_mean_t1',
       'catch_mean_t2', 'catch_sum_t1', 'catch_sum_t2', 'catch_std_t1',
       'catch_std_t2',
       ...
       'game_last_t2', 'event_type_last_t1', 'event_type_last_t2',
       'opponent_clean_last_t1', 'opponent_clean_last_t2',
       'our_score_end_of_point_max_t1', 'our_score_end_of_point_max_t2',
       'their_score_end_of_point_max_t1', 'their_score_end_of_point_max_t2',
       'win_t1'],
      dtype='object', length=145)

In [79]:
df.team_last_t1.unique()

array(['atlantahustle', 'losangelesaviators', 'madisonradicals',
       'chicagowildfire', 'cincinnatirevolution', 'dallasroughnecks',
       'detroitmechanix', 'indianapolisalleycats', 'minnesotawindchill',
       'montrealroyal', 'dcbreeze', 'nashvillenightwatch',
       'charlotteexpress', 'jacksonvillecannons', 'newyorkempire',
       'ottawaoutlaws', 'philadelphiaphoenix', 'pittsburghthunderbirds',
       'raleighflyers', 'austinsol', 'rochesterdragons',
       'sandiegogrowlers', 'sanfranciscoflamethrowers', 'sanjosespiders',
       'seattlecascades'], dtype=object)

In [80]:
df.loc[df.team_last_t1=='newyorkempire']

,game_t1,game_t2,team_id_t1,team_id_t2,catch_mean_t1,catch_mean_t2,catch_sum_t1,catch_sum_t2,catch_std_t1,catch_std_t2,...,game_last_t2,event_type_last_t1,event_type_last_t2,opponent_clean_last_t1,opponent_clean_last_t2,our_score_end_of_point_max_t1,our_score_end_of_point_max_t2,their_score_end_of_point_max_t1,their_score_end_of_point_max_t2,win_t1
game,,,,,,,,,,,,,,,,,,,,,
15_16_10,15_16_10,15_16_10,15,16,8.700000,15.700000,174.0,314.0,6.906138,12.367615,...,15_16_10,Offense,Offense,ottawaoutlaws,newyorkempire,24.0,17.0,17.0,24.0,1
15_16_132,15_16_132,15_16_132,15,16,9.550000,13.200000,191.0,264.0,9.052333,12.833345,...,15_16_132,Offense,Offense,ottawaoutlaws,newyorkempire,23.0,22.0,22.0,23.0,1
15_16_136,15_16_136,15_16_136,15,16,8.944444,14.900000,161.0,298.0,8.292252,15.606679,...,15_16_136,Offense,Offense,ottawaoutlaws,newyorkempire,20.0,20.0,19.0,21.0,1
15_16_144,15_16_144,15_16_144,15,16,9.200000,13.611111,184.0,245.0,7.977600,9.337709,...,15_16_144,Offense,Offense,ottawaoutlaws,newyorkempire,29.0,25.0,25.0,29.0,1
15_16_158,15_16_158,15_16_158,15,16,13.894737,10.750000,264.0,215.0,12.600241,11.350284,...,15_16_158,Offense,Offense,ottawaoutlaws,newyorkempire,28.0,16.0,16.0,28.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15_27_28,15_27_28,15_27_28,15,27,8.350000,12.894737,167.0,245.0,7.450009,12.323847,...,15_27_28,Offense,Offense,torontorush,newyorkempire,16.0,18.0,18.0,16.0,0
15_27_51,15_27_51,15_27_51,15,27,9.842105,9.050000,187.0,181.0,8.610479,8.550008,...,15_27_51,Offense,Offense,torontorush,newyorkempire,19.0,25.0,25.0,19.0,0
15_27_72,15_27_72,15_27_72,15,27,8.947368,11.555556,170.0,208.0,8.396254,8.472948,...,15_27_72,Offense,Offense,torontorush,newyorkempire,14.0,22.0,22.0,14.0,0


#New columns for team - take every 29th column /29 teams
newcols = unstacked.columns[0::29]
newcols
#Remove suffix 
newcols = [e.rstrip('_0') for e in newcols ]

#Add suffix t1 and t2 to columns
cols_t1 = [s + '_t1' for s in newcols]
cols_t2 = [s + '_t2' for s in newcols]
cols_t1[0:5]